In [1]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func, mkdir_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

import eda
from utils import get_categorical_features, get_numeric_features, parallel_process
from feature_engineering import base_aggregation, diff_feature, division_feature, product_feature, cnt_encoding, select_category_value_agg, exclude_feature, target_encoding
from tqdm import tqdm

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

base = utils.read_df_pkl('../input/base0*').set_index(key)

2019-03-19 13:12:33,892 utils 346 [INFO]    [logger_func] start 
100%|██████████| 3/3 [00:00<00:00, 396.81it/s]


In [3]:
"""
Bureau
"""
'''
#========================================================================
# BUREAU
CREDIT_ACTIVE
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64
CREDIT_CURRENCY
currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64
CREDIT_TYPE
Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: CREDIT_TYPE, dtype: int64
#======================================================================== 
'''



def make_cat_features(df, filekey):
    mkdir_func(f'../features/{filekey}')
    train = df[~df[target].isnull()]
    test = df[df[target].isnull()]
    categorical_features = get_categorical_features(df=train, ignore_list=ignore_list)

    #========================================================================
    # Categorical Feature Encode
    #========================================================================
    # Factorize
    logger.info("Factorize Start!!")
    for col in categorical_features:
        for col in categorical_features:
            train[f"lbl_{col}@"], indexer = pd.factorize(train[col])
            test[f"lbl_{col}@"] = indexer.get_indexer(test[col])

    # Count Encoding
    logger.info("Count Encoding Start!!")
    for col in categorical_features:
        train = cnt_encoding(train, col, ignore_list=ignore_list)
        test = cnt_encoding(test, col, ignore_list=ignore_list)

    #========================================================================
    # Categorical Feature Save
    #========================================================================
    for col in train.columns:
        logger.info("Saving Features...")
        if col.count('@'):
            result_train = train[col].values
            result_test = test[col].values
            logger.info(f"COL: {col} | LENGTH: {len(result_train)}")
            utils.to_pkl_gzip(obj=result_train, path=f'../features/{filekey}/train_{col}')
            utils.to_pkl_gzip(obj=result_test, path=f'../features/{filekey}/test_{col}')
            
            
bur = utils.read_df_pkl(path='../input/clean_bur*.p')
display(bur.columns)

100%|██████████| 3/3 [00:00<00:00,  3.37it/s]


Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [ ]:
prefix = 'f001_bur_'
df = bur

org_col_list = df.columns.tolist()

df['AMT_CREDIT_SUM_per_AMT_ANNUITY'] = df['AMT_CREDIT_SUM'] / (df['AMT_ANNUITY'] + 1)

df['AMT_CREDIT_SUM_diff_AMT_CREDIT_DEBT'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_DEBT']
df['AMT_CREDIT_SUM_per_AMT_CREDIT_DEBT'] = df['AMT_CREDIT_SUM'] / (df['AMT_CREDIT_SUM_DEBT'] + 1)

df['AMT_CREDIT_SUM_diff_AMT_CREDIT_DEBT_per_AMT_ANNUITY'] = df['AMT_CREDIT_SUM_diff_AMT_CREDIT_DEBT'] / (df['AMT_ANNUITY'] + 1)

# overdue
df['AMT_CREDIT_SUM_per_SUM_OVERDUE'] = df['AMT_CREDIT_SUM_OVERDUE'] / (df['AMT_CREDIT_SUM'] + 1)
df['AMT_CREDIT_SUM_per_MAX_OVERDUE'] = df['AMT_CREDIT_MAX_OVERDUE'] / (df['AMT_CREDIT_SUM'] + 1)

# limit
df['AMT_CREDIT_SUM_per_LIMIT'] = df['AMT_CREDIT_SUM_LIMIT'] / (df['AMT_CREDIT_SUM'] + 1)
df['AMT_CREDIT_SUM_diff_LIMIT'] = df['AMT_CREDIT_SUM_LIMIT'] - df['AMT_CREDIT_SUM']

# term
df['PLAN_TERM'] = df['DAYS_CREDIT_ENDDATE'] - df['DAYS_CREDIT']
df['FACT_TERM'] = df['DAYS_ENDDATE_FACT'] - df['DAYS_CREDIT']
df['UPDATE_TERM'] = df['DAYS_CREDIT_UPDATE'] - df['DAYS_CREDIT']

# per term
df['AMT_CREDIT_SUM_per_FACT_TERM'] = df['AMT_CREDIT_SUM'] / (df['FACT_TERM'] + 1)
df['AMT_CREDIT_SUM_per_PLAN_TERM'] = df['AMT_CREDIT_SUM'] / (df['PLAN_TERM'] + 1)

# num_list = get_numeric_features(df=df, ignore_list=ignore_list)
# cat_list = get_categorical_features(df=df, ignore_list=[])

aggs = {}

for col in df.columns:
    if col in org_col_list:
        continue
    aggs[col] = ['sum', 'mean', 'max', 'min', 'std']
    
    
df_agg = df.groupby(key).agg(aggs)
new_col_list = [f"{col}-{method}" for col in aggs.keys() for method in aggs[col]]
df_agg.columns = new_col_list

print(df_agg.shape)
display(df_agg.head())

df_agg = base.join(df_agg)
utils.save_feature(df_feat=df_agg, is_train=2, target=target, ignore_list=[key, target], prefix=prefix)

In [4]:
from new_FE import CategoricalEncoding

#========================================================================
# Feature Engineering 
df = bur
cat_enc = CategoricalEncoding(data=df, key=key, target=target, ignore_list=[key, target], base=base)
prefix = 'f001_bur_'
cat_list = get_categorical_features(df, ignore_list)
df_ohe = cat_enc.go_ohe(cat_list=cat_list, method_list=['sum', 'mean'], prefix=prefix)
df_label = cat_enc.go_label(cat_list=cat_list, method_list=['mean', 'std', 'max', 'min'], prefix=prefix)
#========================================================================

100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


(307511,) | ohe_CREDIT_ACTIVE_Active-sum
(307511,) | ohe_CREDIT_ACTIVE_Active-mean
(307511,) | ohe_CREDIT_ACTIVE_Bad debt-sum
(307511,) | ohe_CREDIT_ACTIVE_Bad debt-mean
(307511,) | ohe_CREDIT_ACTIVE_Closed-sum
(307511,) | ohe_CREDIT_ACTIVE_Closed-mean
(307511,) | ohe_CREDIT_ACTIVE_Sold-sum
(307511,) | ohe_CREDIT_ACTIVE_Sold-mean
(307511,) | ohe_CREDIT_CURRENCY_currency 1-sum
(307511,) | ohe_CREDIT_CURRENCY_currency 1-mean
(307511,) | ohe_CREDIT_TYPE_Another type of loan-sum
(307511,) | ohe_CREDIT_TYPE_Another type of loan-mean
(307511,) | ohe_CREDIT_TYPE_Car loan-sum
(307511,) | ohe_CREDIT_TYPE_Car loan-mean
(307511,) | ohe_CREDIT_TYPE_Cash loan _non-earmarked_-sum
(307511,) | ohe_CREDIT_TYPE_Cash loan _non-earmarked_-mean
(307511,) | ohe_CREDIT_TYPE_Consumer credit-sum
(307511,) | ohe_CREDIT_TYPE_Consumer credit-mean
(307511,) | ohe_CREDIT_TYPE_Credit card-sum
(307511,) | ohe_CREDIT_TYPE_Credit card-mean
(307511,) | ohe_CREDIT_TYPE_Interbank credit-sum
(307511,) | ohe_CREDIT_TYPE_Int

100%|██████████| 3/3 [00:00<00:00, 16.15it/s]

[One Hot Encoding] done in 5 s
[Factorize Encoding] done in 0 s


[Aggregation] done in 1 s
(307511,) | label_CREDIT_ACTIVE-mean
(307511,) | label_CREDIT_ACTIVE-std
(307511,) | label_CREDIT_ACTIVE-max
(307511,) | label_CREDIT_ACTIVE-min
(307511,) | label_CREDIT_CURRENCY-mean
(307511,) | label_CREDIT_CURRENCY-std
(307511,) | label_CREDIT_CURRENCY-max
(307511,) | label_CREDIT_CURRENCY-min
(307511,) | label_CREDIT_TYPE-mean
(307511,) | label_CREDIT_TYPE-std
(307511,) | label_CREDIT_TYPE-max
(307511,) | label_CREDIT_TYPE-min
(48744,) | label_CREDIT_ACTIVE-mean
(48744,) | label_CREDIT_ACTIVE-std
(48744,) | label_CREDIT_ACTIVE-max
(48744,) | label_CREDIT_ACTIVE-min
(48744,) | label_CREDIT_CURRENCY-mean
(48744,) | label_CREDIT_CURRENCY-std
(48744,) | label_CREDIT_CURRENCY-max
(48744,) | label_CREDIT_CURRENCY-min
(48744,) | label_CREDIT_TYPE-mean
(48744,) | label_CREDIT_TYPE-std
(48744,) | label_CREDIT_TYPE-max
(48744,) | label_CREDIT_TYPE-min
